In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import sys
import os
import pickle
import librosa
import librosa.display
from IPython.display import Audio
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras

# Taking a Random Audio File from GTZAN Dataset

In [ ]:
audio_data = '/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/classical/classical.00096.wav'
data , sr = librosa.load(audio_data)
print(type(data), type(sr))

In [ ]:
librosa.load(audio_data, sr=45600)

# Playing the Audio File

In [ ]:
import IPython
IPython.display.Audio(data, rate=sr)

# Visualizing Audio :

In [ ]:
plt.figure(figsize=(12, 4))
librosa.display.waveplot(data)
plt.show()

# Plot Spectrogram:

In [ ]:
X = librosa.stft(data)
Xdb = librosa.amplitude_to_db(abs(X))
plt.figure(figsize=(14, 6))
librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')
plt.colorbar()

In [ ]:
plt.figure(figsize=(14, 6))
librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='log')
plt.colorbar()

In [ ]:
from sklearn.preprocessing import normalize
spectral_rolloff = librosa.feature.spectral_rolloff(data+0.01, sr=sr)[0]
plt.figure(figsize=(12, 4))
librosa.display.waveplot(data, sr=sr, alpha=0.8, color = "#102d5a")

In [ ]:
plt.figure(figsize=(12, 4))
librosa.display.waveplot(data, sr=sr, color="#2B4F72")

# Zero-Crossing Rate:

In [ ]:
# Zooming in
n0 = 9000
n1 = 9100
plt.figure(figsize=(14, 5))
plt.plot(data[n0:n1], color="#2B4F72")
plt.grid()

In [ ]:
zero_crossings = librosa.zero_crossings(data[n0:n1], pad=False)
print("The number of zero-crossings is :",sum(zero_crossings))

# Chroma feature

is a powerful tool for analyzing music whose pitches can be meaningfully categorized (often into twelve categories) and whose tuning approximates to the equal-tempered scale. One main property of chroma features is that they capture harmonic and melodic characteristics of music, while being robust to changes in timbre and instrumentation.

![chroma](https://upload.wikimedia.org/wikipedia/commons/thumb/2/25/ChromaFeatureCmajorScaleScoreAudioColor.png/1280px-ChromaFeatureCmajorScaleScoreAudioColor.png?1614745218690)

In [ ]:
chromagram = librosa.feature.chroma_stft(data, sr=sr)
plt.figure(figsize=(15, 5))
librosa.display.specshow(chromagram, x_axis='time', y_axis='chroma', cmap='coolwarm')

# Training Our Model

In [ ]:
%matplotlib inline
from keras import layers
import keras

In [ ]:
df_data = pd.read_csv('../input/gtzan-dataset-music-genre-classification/Data/features_3_sec.csv')
df_data.head()

In [ ]:
df_data.shape

In [ ]:
df_data.dtypes

In [ ]:
df_data.isna().sum()

In [ ]:
df_data.isnull().sum()

In [ ]:
df_data = df_data.drop(labels='filename',axis=1)

In [ ]:
genre_list = df_data.iloc[:, -1]
encoder = LabelEncoder()

In [ ]:
y = encoder.fit_transform(genre_list)

In [ ]:
print(y)

In [ ]:
print(df_data.iloc[:, :-1])

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(np.array(df_data.iloc[:, :-1], dtype = float))

In [ ]:
from sklearn.feature_selection import mutual_info_classif
mutual_info_classif(X,y)

In [ ]:
imp = pd.DataFrame(mutual_info_classif(X,y),index = df_data.iloc[:, :-1].columns)
imp.columns = ['important']
imp.sort_values(by = 'important', ascending = False)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [ ]:
from keras.models import Sequential

In [ ]:
model = Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
classifier = model.fit(X_train,
                    y_train,
                    epochs=180,
                    batch_size=128)

In [ ]:
test_loss, test_acc  = model.evaluate(X_test, y_test, batch_size=128)

In [ ]:
print("The test loss is :",test_loss, "\nThe test accuracy is :",test_acc)